In [15]:
import pandas as pd
import io
import requests
import numpy as np

In [16]:
url="https://raw.githubusercontent.com/JULIELab/EmoBank/master/corpus/emobank.csv"
s=requests.get(url).content
emodata=pd.read_csv(io.StringIO(s.decode('utf-8')))

In [17]:
emodata.head()

,id,V,A,D,text
0,110CYL068_1036_1079,3.00,3.00,3.20,"Remember what she said in my last letter? """
1,110CYL068_1079_1110,2.80,3.10,2.80,If I wasn't working here.
2,110CYL068_1127_1130,3.00,3.00,3.00,".."""
3,110CYL068_1137_1188,3.44,3.00,3.22,Goodwill helps people get off of public assist...
4,110CYL068_1189_1328,3.55,3.27,3.46,Sherry learned through our Future Works class ...


In [18]:
emodata = emodata[['V', 'A', 'text']]
max_v = max(emodata.V)
max_a = max(emodata.A)
emodata.V = emodata.V / max_v
emodata.A = emodata.A / max_a
emodata = emodata.rename(columns={'V': 'Neutral', 'A': 'Position'})
emodata.head()

,Neutral,Position,text
0,0.652174,0.681818,"Remember what she said in my last letter? """
1,0.608696,0.704545,If I wasn't working here.
2,0.652174,0.681818,".."""
3,0.747826,0.681818,Goodwill helps people get off of public assist...
4,0.771739,0.743182,Sherry learned through our Future Works class ...


In [28]:
sentiments = pd.read_csv('Sentiment_Data.csv')
sentiments.head()

,Emoji,Unicode codepoint,Occurrences,Position,Negative,Neutral,Positive,Unicode name,Unicode block
0,😂,0x1f602,14622,0.805101,3614,4163,6845,FACE WITH TEARS OF JOY,Emoticons
1,❤,0x2764,8050,0.746943,355,1334,6361,HEAVY BLACK HEART,Dingbats
2,♥,0x2665,7144,0.753806,252,1942,4950,BLACK HEART SUIT,Miscellaneous Symbols
3,😍,0x1f60d,6359,0.765292,329,1390,4640,SMILING FACE WITH HEART-SHAPED EYES,Emoticons
4,😭,0x1f62d,5526,0.803352,2412,1218,1896,LOUDLY CRYING FACE,Emoticons


In [29]:
max(sentiments.Neutral)

4163

In [19]:
sentiments = sentiments[['Emoji', 'Position', 'Neutral', 'Unicode name']]
sentiments.head()

,Emoji,Position,Neutral,Unicode name
0,😂,0.805101,4163,FACE WITH TEARS OF JOY
1,❤,0.746943,1334,HEAVY BLACK HEART
2,♥,0.753806,1942,BLACK HEART SUIT
3,😍,0.765292,1390,SMILING FACE WITH HEART-SHAPED EYES
4,😭,0.803352,1218,LOUDLY CRYING FACE


In [20]:
# norm Neutral col
max_val = max(sentiments.Neutral)
sentiments.Neutral = sentiments.Neutral / max_val
sentiments.head()

,Emoji,Position,Neutral,Unicode name
0,😂,0.805101,1.000000,FACE WITH TEARS OF JOY
1,❤,0.746943,0.320442,HEAVY BLACK HEART
2,♥,0.753806,0.466491,BLACK HEART SUIT
3,😍,0.765292,0.333894,SMILING FACE WITH HEART-SHAPED EYES
4,😭,0.803352,0.292577,LOUDLY CRYING FACE


In [21]:
def mergeDfs(left, right): 
    left['Emoji'] = 0
    for i in range(len(left)):
        min_dist = 100000
        for j in range(len(right)):
            diff_Position = left.Position[i] - right.Position[j]
            diff_Neutral = left.Neutral[i] - right.Neutral[j]
            euclid_dist = np.power((np.power(diff_Position, 2) + np.power(diff_Neutral, 2)), 0.5)
            if euclid_dist < min_dist:
                min_dist = euclid_dist
                index = j
        left.loc[i, 'Emoji'] = right.loc[index, 'Emoji']  
    return left

In [23]:
df.head()

,Neutral,Position,text,Emoji
0,0.652174,0.681818,"Remember what she said in my last letter? """,♥
1,0.608696,0.704545,If I wasn't working here.,♥
2,0.652174,0.681818,"..""",♥
3,0.747826,0.681818,Goodwill helps people get off of public assist...,😂
4,0.771739,0.743182,Sherry learned through our Future Works class ...,😂


In [ ]:
df = mergeDfs(emodata, sentiments)

In [30]:
df.head()

,Neutral,Position,text,Emoji
0,0.652174,0.681818,"Remember what she said in my last letter? """,♥
1,0.608696,0.704545,If I wasn't working here.,♥
2,0.652174,0.681818,"..""",♥
3,0.747826,0.681818,Goodwill helps people get off of public assist...,😂
4,0.771739,0.743182,Sherry learned through our Future Works class ...,😂


In [25]:
df.to_csv('merged.csv')

In [26]:
df.Emoji.unique()

array(['♥', '😂', '😍', '😘', '😭', '❤'], dtype=object)

In [27]:
sentiments.Emoji.unique()

array(['😂', '❤', '♥', '😍', '😭', '😘', '😊', '👌', '💕', '👏', '😁', '☺', '♡',
       '👍', '😩', '🙏', '✌', '😏', '😉', '🙌', '🙈', '💪', '😄', '😒', '💃', '💖',
       '😃', '😔', '😱', '🎉', '😜', '☯', '🌸', '💜', '💙', '✨', '😳', '💗', '★',
       '█', '☀', '😡', '😎', '😢', '💋', '😋', '🙊', '😴', '🎶', '💞', '😌', '🔥',
       '💯', '🔫', '💛', '💁', '💚', '♫', '😞', '😆', '😝', '😪', '�', '😫', '😅',
       '👊', '💀', '😀', '😚', '😻', '©', '👀', '💘', '🐓', '☕', '👋', '✋', '🎊',
       '🍕', '❄', '😥', '😕', '💥', '💔', '😤', '😈', '►', '✈', '🔝', '😰', '⚽',
       '😑', '👑', '😹', '👉', '🍃', '🎁', '😠', '🐧', '☆', '🍀', '🎈', '🎅', '😓',
       '😣', '😐', '✊', '😨', '😖', '💤', '💓', '👎', '💦', '✔', '😷', '⚡', '🙋',
       '🎄', '💩', '🎵', '➡', '😛', '😬', '👯', '💎', '🌿', '🎂', '🌟', '🔮', '❗',
       '👫', '🏆', '✖', '☝', '😙', '⛄', '👅', '♪', '🍂', '💏', '🔪', '🌴', '👈',
       '🌹', '🙆', '➜', '👻', '💰', '🍻', '🙅', '🌞', '🍁', '⭐', '▪', '🎀', '━',
       '☷', '🐷', '🙉', '🌺', '💅', '🐶', '🌚', '👽', '🎤', '👭', '🎧', '👆', '🍸',
       '🍷', '®', '🍉', '😇', '☑', '🏃', '😿', '│', '💣', '🍺', '▶', '😲